
\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#
#     Subject: NLP Assignment 1       # 
##     Student Name: Nirmal Pregassame
##     Student ID:  R00181587
\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#\#

This notebook is done in Google Colab and represents the work which has been done to complete the Assignment 1 of NLP.
The aim of this assignemnt is to build an **"Error Correction Module"**.
In our model, he will use the output of the OCR system provided by the lecturer.
Then, we will attempt to match each word with a dictionary.
The **"Error Correction Module"** will follow the following procedure:
1. Decontraction of the string
1. Tokenization of the string
1. Check for each words its presence in the dictionary
1. If not, propose a list of proposed words for correction
1. Use a metric distance between the original word and the list of proposed words
1. Select the best words for correction
1. Reset the contraction to its original form
1. Untokenize the string

In the following code, we are trying to unzip the code provided by the lecturer in order to reuse the OCR module

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!unzip -uq "/content/drive/My Drive/Colab Notebooks/Project1NLP_Model.zip" -d "/content/drive/My Drive/Colab Notebooks/Project1NLP_Model"


Mounted at /content/drive


# Decontraction of the string

The concept of the decontraction is to expand the words which are originally contracted in the string we want to correct.
In the following code, a solution for **decontracting the sentence** is proposed.

The idea is to use the module "*Contractions*" from library "*pycontraction*" in a function. This function has 3 parameters:
* *text*: it corresponds to the string which need to be expanded
* *model*: it corresponds to the type of model to use in the api-key of *Contractions*. By default, it is set to "*glove-twitter-100*"
* *precision*: it used to enhance the precision of the expansion. By default it is set to "True"

The process in the function is quite sraight-forward: The model is loaded. Then, the expand_text is applied on the string which generates the end result.

In [2]:
# Installation of api to get pycontractions library
!sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!pip install language-check
!pip install pycontractions

# import the module Contractions from pycontractions
from pycontractions import Contractions


# decontractSentence is a function to expand the contracted word 
def decontractSentence(text, model = 'glove-twitter-25', precision ='True'):
  "Expand all the contracted words to make them proper dict words"
  cont = Contractions(api_key= model)
  cont.load_models()
  print('Sentence to expand:', '"' + text + '"')
  txt_out = list(cont.expand_texts([text], precise= precision))
  
  print('Sentence after expansion:', '"' + txt_out[0] + '"')
  return txt_out



# Tokenization of the string

For the tokenization process, the idea is to split the string into elements (tokens) which can be used as basis of work. Those tokens are generally words (and punctuations).
In the following code, a solution for **Tokenizing the sentence** is proposed.

This time, we use the function "*word_tokenize*" from library "*nltk*". In our code, we build a function which load the string to be tokenized and then tokenize it by applying the nltk function.


In [3]:
import nltk
nltk.download('punkt')

# tokenizeTxt is a function to tokenize a sentence
def tokenizeTxt(text):
  "The function tokenize the sentence stored in text vaiable"
  print('Sentence before tokenization:', '"' + text + '"')
  tokens = nltk.word_tokenize(text)
  print('Sentence after tokenization:', '"' , tokens , '"')
  return tokens



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Dictionary construction

In order to check if a word in a string exists, we need to build a dictionary. This can be done by using the corpus database; more specifically the english version of it since, we should build an english dictionary. Some preprocessing steps were necessary to use the corpus. Indeed, after downloading the corpus, we have to merge all the english texts into one "*all_en.txt*" so that it could be easily read in our function to build the dictionary.

However, Building the dictionary requires few function in order to do it:
1. **ngram generation**: This function is essential to build the ngram at word level so that the dictionary can associate each gram of the word with the word itself
1. **words storage in the dictionnary from the string**: This is the main step of the dictionary. The idea is to process the string/text in order to store each detected word in the dictionary.
1. **word addition**: This function is an open door to add some word if it is not existing in the initial text.

We will go to each of these function to explain how they are used for our dictionary generation.

## ngram generation

The concept of this function is to capture the different grams of the words.
Below, the function accepts 2 parameters:
* *word*: it is the main word for which the gram will be generated
* *n* : it is the gram length. By default, it is set to 2 which means that it is a bigram

This function will be used a multiple time in our code. Indeed, the idea is to associate the grams with the words and count for the number of common grams between 2 words. Typically, the more a non-existing word (a word which has been mispelled) has grams in common with an existing word (from the dictionary), the more likely, those 2 words are close. This can provide a good short-list of words candidate to replace the non-existing word.

In [0]:

# ngrams_gen generate the gram a word ; the gram number is managed by the parameter n
def ngrams_gen(word, n=2):
    "The function generates a list of gram of word. By default, n=2 which means that it is a bigram by default"
    # the set operation is used below to avoid duplicate gram
    ngrams_list = set()
    
    # we go through each letter of word (without the last letters) to generate the gram and create the list
    for i in range(0, len(word) - n + 1):
        ngrams_list.add(word[i:i + n])
        
    # the grams list is sorted befor return
    return sorted(ngrams_list)


## Word storage in the dictionary
​
In this part, the idea is to create a words dictionnary using a text file as input. For the assignment, the file used is the merge of all file in english from the corpus europarl, located at "*/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/europarl/txt/en/all_en.txt*".

During the construction of the dictionary, 3 keys informations are stored. Those are relevant information to process word errors later on. The type of information stored, are:
* *eng_words_dict['txt']*: In this variable, the complete english text of corpus is stored in the form of words list.
* *eng_words_dict['count']*: In this variable, we use the "*Counter*" function from "*collections*" in order to store the number of occurence of a particular word. It is useful at to select the best word for correction when 2 candidate words for replacement of the mispelled word have a tie in the distanc. We can use the number of occurence of the word to propose the best solution.
* *eng_words_dict['gram']*: In this variable, we use the "*defaultdict*" function from "*collections*" in order to make the association of gram and its corresponding list of words in which the gram is present. This is useful, during the error module process, to propose a list of potential words candidate for replacement. Indeed, the idea is to count the number of common gram between the candidate and the original word. The more grams in common they have, the higher is the chance that the candidate is the best candidate for replacement.
* *eng_words_dict['shortw']*:

Below, the function accepts 2 parameters:
* *file*: it represents the file to be read in order to build the dictionary. In the assignment, it is the corpus file 
* *ngram* : it is the gram length. By default, it is set to 2 which means that it is a bigram. This variable is used to build the *eng_words_dict['gram']* associating grams with its list of word.

​
This part represents an essential part for our Error Module Correction.

In [0]:
import collections
import re


# words_dict is a function to create a dictionnary from a text in a file
def words_dict(file, ngram=2):
  
  """4 types classifications are performed for our dictionnary
        1) eng_words_dict['txt']: full text splitted into a list of words the symbot ' @ ' is added to make the sepearation between phrases 
        2) eng_words_dict['count']: each word of the text is associated to its count
        3) eng_words_dict['gram'] : each gram is associated with the list of words containing it 
        4) eng_words_dict['shortw']: list of tuple (<short word>, <short distance>). This tuple are reality for words which length are smaller or equal to the ngram since the ngram dict cannot properly capture them and for mistake detection the gram will be wrong"""
  
  eng_words_dict={}
  print('Read dictionnary')
  # we read the file, make all text to lower case and make a list of words 
  rgxp = re.compile('(<.*>|\.|\?|\!|\;|\(|\))')
  # In this case, we are replacing punctuations and <.*> (which are unecessary introduction of sentence), with the character ' @ ', which is in our case a context delimitation (between sentence for example). 
  # This symbole is useful later on to determine word context in our "context" mode
  eng_words_tmp = rgxp.sub(' @ ',open(file).read())
  # Below, we convert the text into list of words with the addition of context breaking symbole ' @ '.
  eng_words_dict['txt'] = re.findall(r'(\b[A-Za-z_]+\b|\s@\s)', eng_words_tmp.lower())
  
  print('Counting words dictionnary')
  # Thanks to Counter we can summ (and count) all identical words and create a dict with the word associated to its number 
  eng_words_dict['count'] = collections.Counter(eng_words_dict['txt'])
  print('Number of words in dict      : ' , len(eng_words_dict['count'].keys()))
  print('Number of total words in text: ' , sum(eng_words_dict['count'].values()))
  
  # Now we create the correspondance between gram and its words list. (Again, we use the set to avoid the same word in the list)
  eng_words_dict['gram'] = collections.defaultdict(set)
  #
  eng_words_dict['shortw'] = set()
  print('Creating gram collection')
  # we go through all the words found in the text and add them to the list of the corresponding gram
  for w in eng_words_dict['count'].keys():
    for g in ngrams_gen(w,ngram):
      eng_words_dict['gram'][g].add(w)
    if (len(w) <= ngram):
      eng_words_dict['shortw'].add((w,1))
  print('Number of grams: ' , len(eng_words_dict['gram'].keys()))
      
  return eng_words_dict
  

## Word addition in the dictionary
​
In this part, the idea is to give the possibility to add an extra word in the dictionary. Indeed, we assume that not all words are present in the corpus file. Typically, for our assignment, the function below will be used to store the word "Unitek".

During the addition of word in the dictionary, the 4 keys informations from the dictionary are updated to insert this new word.

Below, the function accepts 3 parameters:
* *word*: It represents the word to be added in the dictionnary.
* *w_dict*: It represents the dictionnary which was built previously.
* *ngram* : It is the gram length. By default, it is set to 2 which means that it is a bigram.


In [0]:
#add_word_dict is a function to add a new word in the dictionnary. It is also important to specify the granularity of the gram (by default n=2) to complete our list of words for each gram
def add_word_dict(word,w_dict,ngram=2):
  "Just like for the dictionnary construction above we have to count the number of words in total in the text and then update our words list for the gram"
  print('Adding words in dictionnary:', '"' + word + '"')
  # Just like our word in dictionnary, it is set lower case.
  wl = word.lower()
  # Updating the list of word in the dictionnary with the context breaking symbole ' @ '
  w_dict['txt'].append(' @ ')
  w_dict['txt'].append(wl)
  w_dict['txt'].append(' @ ')
  # Updating the count of the word in the dictionnary
  w_dict['count'][wl] += 1
  # Updating the words list for the gram
  for g in ngrams_gen(wl,ngram):
    w_dict['gram'][g].add(wl)
  if (len(wl) <= ngram):
    w_dict['shortw'].add((wl,1))
  

# List of proposed words

In this part, we will propose a list of acceptable words to replace the non-existing word. Typically, once we find that a word is not in the dictionary, we assume that the word is mispelled. Therefore, a list of word candidates should be proposed. For that, we are counting the number of common gram between the original word and the propsed word. The proposed word should be in an aceptable range of common grams to be part of the word proposed list.

In order to propose this list, we require a few function:
1. **filter for acceptable words**: This function is eensuring the filtering the words within an acceptable range of common grams with the original word.
1. **words list proposition**: This is the main step of the list of words proposition. It ensures the calculation of common grams with the original words to build the list.

We will go to each of these function to explain how they are used for our list of proposed word.

## Filter for accepted Words 
​
The concept here, is to give a threshold above which the candidate words is considered as acceptable to be proposed as replacement for the original word. This function is used a multiple time in our code. Indeed, we used it for building the list of acceptable word after counting the number of common ngram with the original word; but also, we use this function after calculating the word distance between the candidate and the original word to refine the list of candidate.

Below, the function accepts 2 parameters:
* *w*: It represents the tuple (word,value) where *word* is the candidate word for replacement and *value* its weight (It could be for example the number the common gram with the original word or the inverted Levenshtein distance from the original word).
* *threshold*: It represents the limit above which we accepts the candidate word as a potential solution for replacement of the original word.



In [0]:
# filter_accepted_words is a function for filtering the words depending of its closeness to the ref word to correct. A threshold is therefore set to select the best matching words
def filter_accepted_words(w, threshold):
  "The threshold set the limit above which a word is considerated close enough to the ref word for its correction"
  if (w[1])>= threshold:
    return True

## Words List Proposition 
​
This is the main part for proposing a list of candidate words to be a replacement of non-existing word. This function is basically called after the dictionnary is built to compare the non-existing word with the list of existing words. This comparison is done using the "*dictionnary['gram']*" collection. Indeed, the idea is to list all the gram of the non-existing word and for each of them look at the list of words corresponding present in "*dictionnary['gram'][< gram_considered >]*". For each word in the list, we should increment the number of common gram. At the end, we get, for each candidate word, a result of the number of common grams with the non-existing word. Of course, the higher is the number the more probabilty is that this candidate word is the best candidate for replacement.

Below, the function accepts 5 parameters:
* *word*: It represents the original word to be replaced. We basically take this initial word to see what are the grams of this word and then we look for each gram, the list of posibbile words and increment the common gram value for each of those possible words. 
* *w_dict*: It represents the dictionnary. It is used for having the correspondance between gram and words; but also to get the the list of short word w_dict['shortw'].
* *ngram*: It represents the gram length. We use it to calculate the list of the orignal word gram.
* *w_delta_len*: This parameter offer the possibility to select possible words replacement within a certain character range. Typically if this parameter is 1, then we allow the candidate word to have one more or one less character from the initial word: It includes then missing or addition of characters in the init words. By default, this parameter is set to 0. This means that we are considering only character swapping as mistake.
* *similarity*: This parameter should have a value from 0 to 1. It is used to give flexibility for the selection of candidates word. Typically, if set to 0, we accept all possible candidate words which has even only on gram in common with the original word. But if set to 1, we select only the top possible candidate words (The one with the most gram in common with the original word). By default, this parameter is set to 0.5.





In [0]:

      
# words_proposition is a function which provide a list of possible words which could replace the ref word to be corrected
def words_proposition(word, w_dict, ngram=2, w_delta_len=0, similarity=0.5):
  """To propose the list of appropriate words, we count the number of common gram that the word has with the ref word (The word which needs to be corrected):
     the more grams, the word has in common with the ref word, the more likely it is as the best candidate for substitution of ref"""
  
  len_word = len(word)
  
  # if the word is inferior or equal to ngram, the gram process is not suitable for those: We have to take the short word list from dictionary
  # Later on this short word list will be treated the word calculation distance
  if (len_word <= ngram):
    accepted_w_lists = w_dict['shortw']
  else:  # Now the word has a sufficient length to be treated gram by gram
    # Create a dict of words containing common gram with the ref; the value of dict corresponds to the number of common gram
    word_common_gram = collections.defaultdict(int)
    # Below, we have a loop which goes to every gram of ref word (ngrams_gen(word,ngram))
    for g in ngrams_gen(word,ngram):
      # Now, we go to each word of the corresponding ref word common gram
      for w in w_dict['gram'][g]:
        # we pickup only substitute candidate words which has the same length as the ref word (provided a delta word length which is set by default to 0 
        # -- This means by default, we take care only the mispelling and not ommision or addition of letters)  
        if len(w) >= len_word - w_delta_len and len(w) <= len_word + w_delta_len:
          # we count the number of common gram with the ref word
          word_common_gram[w] += 1
    # We sort the candidates word for substitution by descending order (The best candidate is first)
    sort_word_oc = sorted(word_common_gram.items(), key=lambda d: d[1], reverse=True)
    # Below, we try to filter the most relevant words: Typically, we select the best candidate value and we allow a certain margin from the best candidate (by default this margin is set to 50% -- similarity=0.5)
    accepted_threshold = sort_word_oc[0][1] * similarity
    # We use the filter_accepted_words function define earlier to filter, the best candidates
    accepted_w_lists = list(filter(lambda l: filter_accepted_words(l,accepted_threshold), sort_word_oc))
  
  print('Words candidate for correction of word ', '"' + word + '"', ' using ngram technique: ', accepted_w_lists)
  return accepted_w_lists

# Word Distance

In this part, we implement a solution to calculate the distance between the candidate words and the original one. The idea is that once we get the list of words proposition, we have to evaluate "how far" each candidate is from the original word and then return the candidates which are the closest to the original word.

We have 2 possible function to calculate this word distance:
1. **gram distance**: This function return the candidate with the highest ratio of the following coefficient: 2*nb_of_common_grams/(nb_of_gram_from_candidate + nb_of_gram_from_original).
1. **Levenshtein distance**: This function return the candidate with the highest ratio of the following coefficient: 1/lev_distance.

We will go to each of these 2 ways of metric calculation.

## Gram distance  
​
The Gram distance is obtained by the ratio between the common grams number of the 2 considered words and their total number of gram together: 2*nb_of_common_grams/(nb_of_gram_from_candidate + nb_of_gram_from_original). After calculating the distance value of each candidate words and store the tuple (candidate_word,gram_weight), we sort the list from the highest candidate to the lowest. We return then the word candidate with the highest score gram.

Below, the function for gram distance accepts 3 parameters:
* *word*: It represents the original word. This parameter is used to caculate its length (in order to estimate the number of gram in the original word)
* *acc_w_list*: It represents the list of candidate words for replacement.
* *ngram*: It is the gram length. By default, it is set to 2 which means that it is a bigram

**Nota Bene:** Actually, this gram distance is not used in our Assignment. Indeed, the Levenshtein distance was preferred compared to this solution since with the Levenshtein distance, we can add a granularity on the weight between 2 letters. We keep this function since it was the first function implemented to calculate the distance between the candidate and the original word





In [0]:
# The function gram_distance is not used by our proram but is dedicated to calculate the similarity index using a formula with the number of common grams 
def gram_distance(word,acc_w_list,ngram=2):
  "This function using the number of common grams calculated in the function word_proposition, calculate an index of similarity with the ref word"
  selected_words = []
  for w,ng in acc_w_list:
    selected_words.append((w,2*ng/(len(w)-ngram+1+len(word)-ngram+1)))
  
  sort_selected_words = sorted(selected_words, key=lambda d: d[1], reverse=True)
  best_result = sort_selected_words[0][1]
  best_words = list(filter(lambda l: filter_accepted_words(l,best_result), sort_selected_words))
  return best_words
                      


## Levenshtein distance  
​
The Levenshtein distance is here weighted since we add special weight between 2 letters. indeed, special weight are attributed in the case of letters which are considered as very close such as 'i' and 'l' or 'v' and 'u' (The list of letters considered as closed are present in the code). This allows us to add a certain granularity in the calculation of the word distance since a spelling mistake has not the same weight depending on the letter used.

To calculate the Levenshtein distance, we use the "*lev*" function from the library "*weighted_levenshtein*". After calculating the distance value of each candidate words, we store the tuple (candidate_word,1/lev_dist) in our list of best candidate selection "*best_words*". You can note that we are storing the inverse of Levenshtein distance to have the highest score for the closest word candidate. Then, we sort the list from the highest candidate to the lowest and return then the word candidate with the highest score 1/lev.

Below, the function for Levenstein distance accepts 2 parameters:
* *word*: It represents the original word. This parameter is used to caculate the Levenshtein distance with one word of the candidate list.
* *acc_w_list*: It represents the list of candidate words for replacement.


In [10]:
!pip install weighted-levenshtein
import numpy as np
from weighted_levenshtein import lev, osa, dam_lev

# wlev_distance is a function which implements the Levenshtein distance between words (ref word and its candidate for substitution) and build a sorted list of the possible candidates
def wlev_distance(word,acc_w_list):
  "This function uses the weighted_levenshtein library to implement the Levenshtein distance of calculates it for each candidate in comparison of the ref word"
  
  # Initialization of substitute cost for letters which are close
  # some letters like 'u' and 'v' are considered as very close.
  substitute_costs = np.ones((128, 128), dtype=np.float64)
  substitute_costs[ord('u'), ord('v')] = 0.25
  substitute_costs[ord('v'), ord('u')] = 0.25
  substitute_costs[ord('i'), ord('l')] = 0.25
  substitute_costs[ord('l'), ord('i')] = 0.25
  substitute_costs[ord('a'), ord('o')] = 0.25
  substitute_costs[ord('o'), ord('a')] = 0.25
  substitute_costs[ord('c'), ord('o')] = 0.25
  substitute_costs[ord('o'), ord('c')] = 0.25
  substitute_costs[ord('0'), ord('o')] = 0.25
  substitute_costs[ord('o'), ord('0')] = 0.25
  substitute_costs[ord('g'), ord('q')] = 0.25
  substitute_costs[ord('q'), ord('g')] = 0.25
  substitute_costs[ord('r'), ord('n')] = 0.25
  substitute_costs[ord('n'), ord('r')] = 0.25
  substitute_costs[ord('s'), ord('5')] = 0.25
  substitute_costs[ord('5'), ord('s')] = 0.25
  
  # selected_words will store a tuple of candiate word and it Levenshtein distance
  selected_words = []
  # the ref word is set to lower case just like all the words in the dictionnary  
  word_l = word.lower()
  # We go through the candidate words list generated previously (in word_proposition for example).
  for w,ng in list(acc_w_list):
    # Levenshtein distance is applied from ref word to the one in the list (we store in reality the inverse value of Lev distance)
    lev_dist = lev(word_l, w, substitute_costs=substitute_costs)
    selected_words.append((w,1/lev_dist))
  
  # We sort the candidates word for substitution by descending order -- The value corresponds to the inv of lev distance (The best candidate is first)
  sort_selected_words = sorted(selected_words, key=lambda d: d[1], reverse=True)
  # Below, we try to filter the most relevant words: Typically, we select only the top candidate (the ones with the highest score)
  best_result = sort_selected_words[0][1]
  # We use again the filter_accepted_words function define earlier to select only the top candidates
  best_words = list(filter(lambda l: filter_accepted_words(l,best_result), sort_selected_words))
  
  print('Words Top candidates for correction of word ', '"' + word + '"', '  after Levenshtein distance calculation: ', best_words)
  # We retrun only the candidates with the best score
  return best_words


# Best Word Correction Solution

In this part, we implement a solution in order to solve tie between multiple candidate word for replacement. Typically, after calculating the word distance (with the Levenshtein distance for example), we might have more than one candidate which have the best distance. In his case, we should choose the appropriate word to return the best word. For this, we have 2 possibilities to choose the best word:
* **occurence**: The idea is to return the candidate word which has the most occurence in the text (corpus text). We use in this case the element dict['count'] of the corpus dictionary which has the correspondance between a word and its occurence in the text.
* **contextual**: The idea is to return the candidate word for which the context in the text is closer to the context in the sentence to be corrected. We use in this case the element dict['txt'] of the corpus dictionary and look the word before and after the candidate word and compare them with the word before and after the original word to be corrected in its sentence. If one of those words match then we increment the context_weight. It is in a way a simpified **bigram** process at sentence level to determine the context. We return the word which has the highest context weight.

The function below support 5 parameters:
1. *t_position*: It represents the position of the word to be corrected in the tokenized list of the original sentence. This parameter is only used when we selct the best word solution in **contextual** mode as we have to look before and after the word to see its context. 
1. *t_string*: It represents the tokenized list of the original sentence. This parameter is only used when we selct the best word solution in **contextual** mode just like the parameter above.
1. *wordsList*: It represents the list of candidate words to replace the original word. This list comes from the word distance calculation (Levenshtein distance) and the idea is to select the best word for replacement using the **occurence** mode or the **contextual** mode to solve the tie between them after calculating the distance. if there is only one word in the list, we just have to return that word.
1. *wordsDict*: It represents the dictionary we have built earlier. We use this dictionary for checking the occurence of the word in **occurence** mode or to see the context before and after a candidate word in **contextual** mode.
1. *selectType*: It represents the selection type of the best word candidate for replacement. It can be either "*occurence*" or "*contextual*". 


In [0]:
import collections
import numpy as np

# After calculating the Levenshtein distance, we may have multiple candidates word. We then have to take one of them.
# The idea of best_word_correction is to select the candidate with the highest occurance in the dictionnary.
def best_word_correction(t_position, t_string, wordsList, wordsDict, selectType='contextual'):
  "If there are many candidates, The function return the best candidate accordingly to its occurance in the text used for the dict.  "
  
  # if there is only one possibility then return this candidate
  if len(wordsList) == 1:
    best_word = wordsList[0][0]
  # otherwise, we have to look the number of occurence of the words in the text
  elif selectType=='occurence':
    selected_words = []
    # for each candidate we see its occurence ...
    for w,ng in wordsList:
      # ... and store them in the selected_words as tuple
      selected_words.append((w,wordsDict['count'][w]))
    # At the end we need just to sort in decreasing order (the ones with most occurence is 1st in the list)
    sort_selected_words = sorted(selected_words, key=lambda d: d[1], reverse=True)
    print('Words Top candidates for correction of word ', '"' + t_string[t_position] + '"', '  after using "occurence" mode: ', sort_selected_words)
    # We return the top word as a solution for the best word.
    best_word = sort_selected_words[0][0]
    
  elif selectType=='contextual':
    # Converting the dict into numpy array
    np_dict = np.array(dict_corpus['txt'])
    # we extract the word before and after the original word for comparison
    word_before = t_string[t_position-1].lower() if t_position > 0 else ' @ ' 
    word_after = t_string[t_position+1].lower() if t_position < len(t_string)-1 else ' @ ' 
    context_weight = collections.defaultdict(int)
    # for each candidate we see its context ...
    for w,ng in wordsList:
      # We extract all the position of the candidate word in the dict text
      np_ind_w = np.where(np_dict == w)[0]
      # then , we check the word before and after the candidate word
      np_context_before = np_dict[np_ind_w-1]
      np_context_after = np_dict[np_ind_w+1]
      # we compare them with the words before and after the original one and see how many are they
      context_weight_before = len(np_context_before[np_context_before==word_before])
      context_weight_after = len(np_context_after[np_context_after==word_after])
      # the context weight is calculated with the number of similar (before and after) words in comparison to the ones with the original words 
      context_weight[w] = context_weight_before + context_weight_after
    # We sort the candidates word for substitution by descending order depending on the weight of the context (The best candidate is first)
    sort_context_weight = sorted(context_weight.items(), key=lambda d: d[1], reverse=True)
    print('Words Top candidates for correction of word ', '"' + t_string[t_position] + '"', '  after using "context" mode: ', sort_context_weight)
    best_word = sort_context_weight[0][0]
    
  else:
    best_word = wordsList[0][0]
    
  print('Best word for replacement: ', best_word)
  return best_word


# Sentence Correction

This part is the main process of our **Error Correction Module**. Indeed, the concept is to go through each word of the original given string and propose a correction of it if it is not in the dictionnary. The full process of this Sentence Correction is described below:
1. Go to every token of the original sentence string '*t*'.
1. Set the token to lower case '*tl*'
1. If '*tl*' is a word and if it is not present in the dict, check the words correction proposition with the function '*words_proposition*'
1. Then, use the Levenshtein distance between the words candidate list and the original word thanks to the function '*wlev_distance*'
1. After, we select the best choice of word replacement using the function '*best_word_correction*' in contextual mode
1. All the corrected word (original or corrected) is appended into the list which forms the corrected sentence from the original string.

The function below accepts 6 parameters:
* *token_sentence*: It represents the tokenized original string sentence which needs to be corrected. We basically go through each token of the string to check if it is present in the dictionary and eventually propose a correction if not. 
* *w_dict*: It represents the dictionnary build initially from a reference text (like corpus). It is the base of the check of the existence of the word
* *ngram*: It represents the gram length. We use it to calculate the list of the orignal word gram.
* *w_len_mode*: This parameter offer the possibility to select possible words replacement within a certain character range. Typically if this parameter is 1, then we allow the candidate word to have one more or one less character from the initial word: It includes then missing or addition of characters in the init words. By default, this parameter is set to 0. This means that we are considering only character swapping as mistake.
* *similarity*: This parameter should have a value from 0 to 1. It is used to give flexibility for the selection of candidates word. Typically, if set to 0, we accept all possible candidate words which has even only on gram in common with the original word. But if set to 1, we select only the top possible candidate words (The one with the most gram in common with the original word). By default, this parameter is set to 0.5.
* *selectType*: It represents the selection type of the best word candidate for replacement. It can be either "*occurence*" or "*contextual*". "*occurence*" will propose the word for replacement based on its occurence in the ref text (corpus) whereas, "*contextual*" witll propose the word based on the context of the word.



In [0]:
import re

# sentence_correction is the function which goes through each word of the sentence to check it existence in the dict and eventually propose a correction
def sentence_correction(token_sentence, w_dict, ngram, w_len_mod=0, similarity=0.5, selectType='contextual'):
  "This function is the body of the code, since it evaluates for each word of a given sentence its existence in the dict and apply a suggestion of correction "
  
  # corrected_sentence is the final tokenized sentence after replacement of non-existing word
  corrected_sentence = []
  t_position = 0
  
  # we go through each element of the original reference sentence. (the element could be a non-word such as a punctuation)
  for t in token_sentence:
    # we convert the element to lower case
    tl = t.lower()
    # And we check if this element is a potential word and if it is in the dict
    if re.match(r"\w+",tl) and w_dict['count'][tl] == 0:
      # if the word is not in the list, we apply the words_suggestion function to give potential words candidate for replacement
      words_suggestion = words_proposition(tl, w_dict, ngram, w_len_mod, similarity)
      # we refine the suggested candidates by applying the Lev distance
      words_list = wlev_distance(tl,words_suggestion)
      # we return the best choice according to the word occurence in the dict
      #corrected_word = best_word_correction(words_list,w_dict['count'])
      corrected_word = best_word_correction(t_position, token_sentence, words_list, w_dict, selectType)
      # we append this word in the corrected sentence
      corrected_sentence.append(corrected_word)
    else:
      # if the element is a non-word or if it is in the dict, reuse it
      corrected_sentence.append(t)
    t_position += 1
  
  print('Tokenized and expanded sentence before correction: ', token_sentence)
  print('Tokenized and expanded sentence after correction: ', corrected_sentence)
  
  # return the sentence with the proposed correction
  return corrected_sentence
      
      
    

# Contraction of the solution string

The concept of the solution string contraction is to contract the words which were originally contracted in the initial string to keep its original form. But in our code this part is not just used to contract word; it also tries to keep the same formatting as the initial string. Typically, if the word has an upper case, we try to keep this uppercase.

In the following code, a solution for **Contracting of the solution sentence** is proposed.

The idea is to use the the tokenized initial string (with contraction) and the new soution string (without contraction) and then make a comparison between them to check which tokens ar different. Typically, if one token of original string starts with the character "'", and not in the solution string, then we have a case of contracted word.

We do the same process for word with upper case and lower case, and we can have at the end a solution string which keeps the same formatting as the original sentence. 

The function for this contraction function has 2 parameters:
* *new_st*: it corresponds to the new solution string which need to be worked to get the same formatting as the original string.
* *init_st*: it corresponds tto the original string which is the model of formatting to be used.

In [0]:
import re

# contract_to_init_s is a function which set again the contracted words to its original state and keep the capital letters when it can
def contract_to_init_s(new_st, init_st):
  "This function takes as input the original sentence tokenized with the tokenized new solution and then checks what word have been contracted and which one as an upercase an try to keep it"
  
  # result_token is the final sentence which keeps upper case and contraction
  result_token = []
  
  # In this loop we are going through each element of the tokenized solution in parallel with the tokenized original sentence
  for n,i in zip(new_st, init_st):
    if re.match(r"^'\w+",i):             # We check for word starting with "'" following by at least a letter word which means that it is a contracted word
      result_token.append(i)             # if it is a contracted word we add it in the result
    elif(i.lower() == n and not i.islower()): # then we check if there is a capital letter in the original word which is identical word as the solution word
      result_token.append(i)                  # if it is the case we keep the original word
    elif(len(i) == len(n) and not i.islower()): # if the length of original and new word are same, we keep the upper case to the same position letter for the new word (here new word and old word are not same -- it has been corrected)
      n_tmp=[]
      for k in range(len(i)):
        if i[k].isupper():
          n_tmp.append(n[k].upper())
        else:
          n_tmp.append(n[k])
      result_token.append("".join(n_tmp))
    elif(not i.islower()):                 # if the length of new word and original word is different (and they are not identical and does not have same lengh), we keep only the 1st upper case letter if it is in original word
      n_tmp=[]
      if i[0].isupper():
        n_tmp.append(n[0].upper())
      else:
        n_tmp.append(n[0])
      for k in range(1, len(n)):
        n_tmp.append(n[k])
      result_token.append("".join(n_tmp))
    else:
      result_token.append(n)              # End result of sentence after keeping contraction and upper case.

  print('Contracted and formatted sentence after correction: ', result_token)
  
  # return the result of the solution
  return result_token

# Detokenization of the solution string

For the detokenization process, the idea is to merge all the tokens to get at the end a proper string sentence.

In the following code, a solution for **Detokenizing of the solution sentence** is proposed.

This time, we use the function "*TreebankWordDetokenizer.detokenize*" from library "*nltk.tokenize.treebank*". In our code, we build a function which load the string to be detokenized and then detokenize it by applying the nltk.tokenize.treebank function.


In [0]:
from nltk.tokenize.treebank import TreebankWordDetokenizer

# detokenizeTxt is a function which revert the tokenisation operation (it untokenizes)
def detokenizeTxt(t):
  "this function uses the nltk.tokenize.treebank package to untokenize the sentence"
  detoken = TreebankWordDetokenizer().detokenize(t)
  
  print('Untokenized solution sentence: ', detoken)
  
  # return the untokenized sentence
  return detoken

# Error Correction Module

**This section corresponds to the final solution of our assignement: "*An Error Correction Module*" is proposed.**

**The function calls the different function above in order to return at the end the corrected sentence**


In [0]:

def Error_Correction_Module(sentence, dict_ref, ngram, word_delta_len_modulation=0, word_similarity_acceptation=0.5, word_selection_type='contextual'):
  d_sentence = decontractSentence(sentence)          # the sentence is decontracted 
  init_sentence = tokenizeTxt(sentence)              # we tokenize the original sentence to use it at the end for contracting the solution sentence
  tokenized_sentence = tokenizeTxt(d_sentence[0])    # then, it is tokenized
  new_sentence = sentence_correction(tokenized_sentence, dict_ref, ngram, word_delta_len_modulation, word_similarity_acceptation, word_selection_type) # we launch our sentence correction solution
  contracted_sentence = contract_to_init_s(new_sentence, init_sentence) # then we contract the obtained solution to keep as much as possible the sam contraction and upper case letter thant the original sentence
  final_sentence=detokenizeTxt(contracted_sentence)                     # final result sentence is ready
  return final_sentence


### Building Dictionnary : Corpus database

After implementing the Error Correction module, it is time to do some test on our module.

We should first build our dictionary using the corpus text in english. For that, we use the function "*words_dict*" implemented previously.


In [16]:
corpus_file = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/europarl/txt/en/all_en.txt'  # now we will consider the corpus file as input  

dict_corpus = words_dict(corpus_file, 2)                              # we build our dictionnary
  

Read dictionnary
Counting words dictionnary
Number of words in dict      :  78600
Number of total words in text:  58440476
Creating gram collection
Number of grams:  675


### Error Correction Module Test

Below, we will try different sentence to see the behavior of the Error Correction Module.

The idea is to test 3 different sentences (with some mispelled words) and see the proposed solution of our Error Correction Module. The 3 sentences are:

1. "*He ta**b**es his time to pay his ta**b**es*" : For this sentence the expected result should be "He ta**k**es his time to pay his ta**x**es". 

1. "*He **k**s with **k**s*" : For this sentence the expected result should be "He **i**s with **u**s". 

1. "*I wa**f**ted outside for hours and I wa**f**ted my time*" : For this sentence the expected result should be "I wa**i**ted outside for hours and I wa**s**ted my time". 

In the 3 above cases, we have a very interesting sentence to test since each of them have identical mispelled words for 2 different existing words. Typically,
* For the 1st sentence, we have "*tabes*" for respectively "*takes*" and "*taxes*"
* For the 2nd sentence, we have "*ks*" for respectively "*is*" and "*us*"
* For the 3rd sentence, we have "*wafted*" for respectively "*waited*" and "*wasted*"

In those cases, the Levenshtein distance between the mispelled word and the correct one is same for both corrected words. Therefore, we should get multiple solutions after the Levenshtein word distance calculation with the function "*wlev_distance*".  The function "*best_word_correction*" should therefore manage to determine what is the optimal solution between those different possible correct words using the 2 possible modes we have implemented the "*occurence*" mode and the "*conceptual*" mode. 

The 2nd sentence is also stressing the mispelling of a short words like "*is*" or "*us*". Indeed, we are using bigram matching pattern to make our proposition of corrected words which in theory should not work not very well for short words.

Our work is to check now how the Error Module is solving those mistakes: 


In [17]:

# below we apply our methodology for word correction in various test sentences

orig_sentence= 'He tabes his time to pay his tabes'
orig_sentence2= 'He ks with ks'
orig_sentence3= 'I wafted outside for hours and I have wafted my time'
  
ngram = 2                                                             # we are considering the bigram for our solution
word_delta_len_modulation = 0                                         # we are not accepting any delta letter more or less than the original word (only spelling mistake is considered)
word_similarity_acceptation = 0.5                                     # we only propose a similarity of 50% from the best proposition for our range of proposed word for correction

print('')
print('###### TEST ON 1st SENTENCE: ' + '"' + orig_sentence + '"')
print('## On occurence mode' )
result_sentence1_occ = Error_Correction_Module(orig_sentence, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'occurence')
print('')
print('## On contextual mode' )
result_sentence1_context = Error_Correction_Module(orig_sentence, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'contextual')
print('')
print('## Final Result' )
print("Original sentence 1:                     ", orig_sentence)
print("Result of sentence 1 in occurence mode:  ", result_sentence1_occ)
print("Result of sentence 1 in contextual mode: ", result_sentence1_context)

print('')
print('')
print('###### TEST ON 2nd SENTENCE: ' + '"' + orig_sentence2 + '"')
print('## On occurence mode' )
result_sentence2_occ = Error_Correction_Module(orig_sentence2, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'occurence')
print('')
print('## On contextual mode' )
result_sentence2_context = Error_Correction_Module(orig_sentence2, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'contextual')
print('')
print('## Final Result' )
print("Original sentence 2:                     ", orig_sentence2)
print("Result of sentence 2 in occurence mode:  ", result_sentence2_occ)
print("Result of sentence 2 in contextual mode: ", result_sentence2_context)

print('')
print('')
print('###### TEST ON 3rd SENTENCE: ' + '"' + orig_sentence3 + '"')
print('## On occurence mode' )
result_sentence3_occ = Error_Correction_Module(orig_sentence3, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'occurence')
print('')
print('## On contextual mode' )
result_sentence3_context = Error_Correction_Module(orig_sentence3, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, 'contextual')
print('')
print('## Final Result' )
print("Original sentence 3:                     ", orig_sentence3)
print("Result of sentence 3 in occurence mode:  ", result_sentence3_occ)
print("Result of sentence 3 in contextual mode: ", result_sentence3_context)




###### TEST ON 1st SENTENCE: "He tabes his time to pay his tabes"
## On occurence mode


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Sentence to expand: "He tabes his time to pay his tabes"
Sentence after expansion: "He tabes his time to pay his tabes"
Sentence before tokenization: "He tabes his time to pay his tabes"
Sentence after tokenization: " ['He', 'tabes', 'his', 'time', 'to', 'pay', 'his', 'tabes'] "
Sentence before tokenization: "He tabes his time to pay his tabes"
Sentence after tokenization: " ['He', 'tabes', 'his', 'time', 'to', 'pay', 'his', 'tabes'] "
Words candidate for correction of word  "tabes"  using ngram technique:  [('tabet', 3), ('babes', 3), ('stabi', 2), ('sabet', 2), ('haben', 2), ('abeed', 2), ('abera', 2), ('tabou', 2), ('habet', 2), ('abeto', 2), ('stabs', 2), ('abbey', 2), ('zabel', 2), ('haber', 2), ('labem', 2), ('taboo', 2), ('abbed', 2), ('label', 2), ('abeba', 2), ('babel', 2), ('table', 2), ('jaber', 2), ('obese', 2), ('beset', 2), ('berta', 2), ('besis', 2), ('pdbes', 2), ('beste', 2), ('beata', 2), ('besim', 2), ('benes', 2), ('vibes', 2), ('kubes', 2), ('cubes', 2), ('gibes', 

### Explanation of the result

The Test of our ECM (Error Correction Module) gives interesting result:

1. For the 1st sentence, "*He ta**b**es his time to pay his ta**b**es*" , we get: 
   * In "*occurence*" mode : "*He ta**k**es his time to pay his ta**k**es*"
   * In "*contextual*" mode: "*He ta**k**es his time to pay his ta**x**es*".
   
   As expected, the "*contextual*" mode gives a better result than the "*occurence*" mode. Indeed, in the "*occurence*" mode, we simply take the word which has the highest occurence in the corpus text to solve the Levenshtein distance tie between the words "*takes*" and "*taxes*" as replacement of "*tabes*". Despite of being faster than the "*contextual*" mode, this strategy has its limit since it does not see the context of the word in the sentence and apply the replacement without "knowing" with the requested words needs to be a "noun" or a "verb".
   
   The "*contextual*" mode removes this limitation; but as a drawback, the process is a little bit longer, since it has to go throuw all the words list in the dictionary dict_corpus['txt'] to determine the context of each candidate before and after.
   
1. For the 2nd sentence, "*He **k**s with **k**s*" , we get: 
   * In "*occurence*" mode : "*He **i**s with **i**s*"
   * In "*contextual*" mode: "*He **i**s with **u**s*".
   
   Again, the "*contextual*" mode gives a better result than the "*occurence*" mode for the same reason as above.
   
   But actually, the interesting part here is whatever the context ("*occurence*" and "*contextual*"), the ECM is able to catch small words mispelling mistake (words with less than 3 characters). Indeed, as we are using bigram tokens to check the similarity between the candidate words and the mispelled words. But in our "*words_proposition*" function, we implement special case for short words mispelling and we are not using anymore the bigram tokens but the dict_corpus['shortw'] list. This is the reason why, our ECM is able to handle short words mispelling.
   
1. For the 3rd sentence, "*I wa**f**ted outside for hours and I wa**f**ted my time*", we get: 
   * In "*occurence*" mode : "*I wa**n**ted outside for hours and I wa**n**ted my time*"
   * In "*contextual*" mode: "*I wa**n**ted outside for hours and I wa**n**ted my time*".
   
   Here, we can see the limitation of our ECM model. Indeed, the expected sentence should have been "*I wa**i**ted outside for hours and I wa**s**ted my time*"
   

#### Limitation Explanation
   
   This limitation is visible even in our "*contextual*" mode. The reason is that in this sentence the 2 expected correct words ("waited" and "wasted") and the proposed word ("wanted") are very similar in their context: They are all 3 verbs with only one Levenshtein distance between them. Their usage is approximatively identical as the 3 verbs are used following a noun and they are transitive verbs which means they are also automatically followed by noun. Therefore determining the context is not that simple.
   
   In order, to remove thsi limitation, we should have taken a wider context. Instead of using bigram context (word before and after targeted word), we should use trigram or 4-gram context (respectively 2 words before and after; and 3 words before and after) to widen the context. A larger corpus could also be a little bit helpful. However, in that case, the ECM process would have been extremely heavy and very long to produce a result.
   


# Error Correction Module applied after the OCR

In this section, we are testing the Error Correction Module. For that we use the output of the OCR system provided by the lecturer and then try to correct the output word.

Most of the part below has not been implemented by the student. The complete OCR module has been reused from the lecturer program. Only the Main section of the program has been modified a litle bit to support the **Error Correction Module** after collecting the output of OCR module.

In [18]:
import random
import sys
import numpy as np
import cv2
import editdistance
import tensorflow as tf



#ExpDir = '/Users/haithem.afli/Desktop/CIT/Research/IndustryProjects/Unitek/NN_OCR/run_model/'
#ExpDir = r'C:\Users\npregass\OneDrive - Qualcomm\Documents\Masters AI\NLP\Assignement 1\Project1NLP_Model\Project1NLP_Model'
ExpDir = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/Project1NLP_Model/'
#print ('Experiment Dir is:' + ExpDir)
#modelDir = r'C:\Users\npregass\OneDrive - Qualcomm\Documents\Masters AI\NLP\Assignement 1\Project1NLP_Model\Project1NLP_Model/model/'
modelDir = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/Project1NLP_Model/model/'
#modelDir = '/Users/haithem.afli/Desktop/CIT/Research/IndustryProjects/Unitek/NN_OCR/run_model/model/'
#print ('Model is in:' + modelDir)
#You need to change this path with your local path to the saved model I provides with the code

class FilePaths:
  "filenames and paths to data"
  fnCharList = modelDir+'charList.txt'
  fnAccuracy = modelDir+'accuracy.txt'
  fnInfer = ExpDir+'unitek12.png'

    

class Batch:
  "batch containing images and ground truth texts"
  def __init__(self, gtTexts, imgs):
    self.imgs = np.stack(imgs, axis=0)
    self.gtTexts = gtTexts

class DecoderType:
  BestPath = 0
  BeamSearch = 1
  WordBeamSearch = 2
        
def preprocess(img, imgSize, dataAugmentation=False):
  "put img into target img of size imgSize, transpose for TF and normalize gray-values"

  # there are damaged files in IAM dataset - just use black image instead
  if img is None:
    img = np.zeros([imgSize[1], imgSize[0]])

  # increase dataset size by applying random stretches to the images
  if dataAugmentation:
    stretch = (random.random() - 0.5) # -0.5 .. +0.5
    wStretched = max(int(img.shape[1] * (1 + stretch)), 1) # random width, but at least 1
    img = cv2.resize(img, (wStretched, img.shape[0])) # stretch horizontally by factor 0.5 .. 1.5
  
  # create target image and copy sample image into it
  (wt, ht) = imgSize
  (h, w) = img.shape
  fx = w / wt
  fy = h / ht
  f = max(fx, fy)
  newSize = (max(min(wt, int(w / f)), 1), max(min(ht, int(h / f)), 1)) # scale according to f (result at least 1 and at most wt or ht)
  img = cv2.resize(img, newSize)
  target = np.ones([ht, wt]) * 255
  target[0:newSize[1], 0:newSize[0]] = img

  # transpose for TF
  img = cv2.transpose(target)

  # normalize
  (m, s) = cv2.meanStdDev(img)
  m = m[0][0]
  s = s[0][0]
  img = img - m
  img = img / s if s>0 else img
  return img

def train(model, loader):
  "train NN"
  epoch = 0 # number of training epochs since start
  bestCharErrorRate = float('inf') # best valdiation character error rate
  noImprovementSince = 0 # number of epochs no improvement of character error rate occured
  earlyStopping = 5 # stop training after this number of epochs without improvement
  while True:
    epoch += 1
    print('Epoch:', epoch)

    # train
    print('Train NN')
    loader.trainSet()
    while loader.hasNext():
      iterInfo = loader.getIteratorInfo()
      batch = loader.getNext()
      loss = model.trainBatch(batch)
      print('Batch:', iterInfo[0],'/', iterInfo[1], 'Loss:', loss)

    # validate
    charErrorRate = validate(model, loader)
    
    # if best validation accuracy so far, save model parameters
    if charErrorRate < bestCharErrorRate:
      print('Character error rate improved, save model')
      bestCharErrorRate = charErrorRate
      noImprovementSince = 0
      model.save()
      open(FilePaths.fnAccuracy, 'w').write('Validation character error rate of saved model: %f%%' % (charErrorRate*100.0))
    else:
      print('Character error rate not improved')
      noImprovementSince += 1

    # stop training if no more improvement in the last x epochs
    if noImprovementSince >= earlyStopping:
      print('No more improvement since %d epochs. Training stopped.' % earlyStopping)
      break


def validate(model, loader):
  "validate NN"
  print('Validate NN')
  loader.validationSet()
  numCharErr = 0
  numCharTotal = 0
  numWordOK = 0
  numWordTotal = 0
  while loader.hasNext():
    iterInfo = loader.getIteratorInfo()
    print('Batch:', iterInfo[0],'/', iterInfo[1])
    batch = loader.getNext()
    recognized = model.inferBatch(batch)
    
    print('Ground truth -> Recognized')  
    for i in range(len(recognized)):
      numWordOK += 1 if batch.gtTexts[i] == recognized[i] else 0
      numWordTotal += 1
      dist = editdistance.eval(recognized[i], batch.gtTexts[i])
      numCharErr += dist
      numCharTotal += len(batch.gtTexts[i])
      print('[OK]' if dist==0 else '[ERR:%d]' % dist,'"' + batch.gtTexts[i] + '"', '->', '"' + recognized[i] + '"')
  
  # print validation result
  charErrorRate = numCharErr / numCharTotal
  wordAccuracy = numWordOK / numWordTotal
  print('Character error rate: %f%%. Word accuracy: %f%%.' % (charErrorRate*100.0, wordAccuracy*100.0))
  return charErrorRate

modelDir = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/Project1NLP_Model/model/'

class Model: 
  "minimalistic TF model for HTR"

  # model constants
  batchSize = 50
  imgSize = (128, 32)
  maxTextLen = 32

  def __init__(self, charList, decoderType=DecoderType.BestPath, mustRestore=False):
    "init model: add CNN, RNN and CTC and initialize TF"
    self.charList = charList
    self.decoderType = decoderType
    self.mustRestore = mustRestore
    self.snapID = 0

    # CNN
    self.inputImgs = tf.placeholder(tf.float32, shape=(Model.batchSize, Model.imgSize[0], Model.imgSize[1]))
    cnnOut4d = self.setupCNN(self.inputImgs)

    # RNN
    rnnOut3d = self.setupRNN(cnnOut4d)

    # CTC
    (self.loss, self.decoder) = self.setupCTC(rnnOut3d)

    # optimizer for NN parameters
    self.batchesTrained = 0
    self.learningRate = tf.placeholder(tf.float32, shape=[])
    self.optimizer = tf.train.RMSPropOptimizer(self.learningRate).minimize(self.loss)

    # initialize TF
    (self.sess, self.saver) = self.setupTF()

      
  def setupCNN(self, cnnIn3d):
    "create CNN layers and return output of these layers"
    cnnIn4d = tf.expand_dims(input=cnnIn3d, axis=3)

    # list of parameters for the layers
    kernelVals = [5, 5, 3, 3, 3]
    featureVals = [1, 32, 64, 128, 128, 256]
    strideVals = poolVals = [(2,2), (2,2), (1,2), (1,2), (1,2)]
    numLayers = len(strideVals)

    # create layers
    pool = cnnIn4d # input to first CNN layer
    for i in range(numLayers):
      kernel = tf.Variable(tf.truncated_normal([kernelVals[i], kernelVals[i], featureVals[i], featureVals[i + 1]], stddev=0.1))
      conv = tf.nn.conv2d(pool, kernel, padding='SAME',  strides=(1,1,1,1))
      relu = tf.nn.relu(conv)
      pool = tf.nn.max_pool(relu, (1, poolVals[i][0], poolVals[i][1], 1), (1, strideVals[i][0], strideVals[i][1], 1), 'VALID')

    return pool


  def setupRNN(self, rnnIn4d):
    "create RNN layers and return output of these layers"
    rnnIn3d = tf.squeeze(rnnIn4d, axis=[2])

    # basic cells which is used to build RNN
    numHidden = 256
    cells = [tf.contrib.rnn.LSTMCell(num_units=numHidden, state_is_tuple=True) for _ in range(2)] # 2 layers

    # stack basic cells
    stacked = tf.contrib.rnn.MultiRNNCell(cells, state_is_tuple=True)

    # bidirectional RNN
    # BxTxF -> BxTx2H
    ((fw, bw), _) = tf.nn.bidirectional_dynamic_rnn(cell_fw=stacked, cell_bw=stacked, inputs=rnnIn3d, dtype=rnnIn3d.dtype)
                  
    # BxTxH + BxTxH -> BxTx2H -> BxTx1X2H
    concat = tf.expand_dims(tf.concat([fw, bw], 2), 2)
                  
    # project output to chars (including blank): BxTx1x2H -> BxTx1xC -> BxTxC
    kernel = tf.Variable(tf.truncated_normal([1, 1, numHidden * 2, len(self.charList) + 1], stddev=0.1))
    return tf.squeeze(tf.nn.atrous_conv2d(value=concat, filters=kernel, rate=1, padding='SAME'), axis=[2])
    

  def setupCTC(self, ctcIn3d):
    "create CTC loss and decoder and return them"
    # BxTxC -> TxBxC
    ctcIn3dTBC = tf.transpose(ctcIn3d, [1, 0, 2])
    # ground truth text as sparse tensor
    self.gtTexts = tf.SparseTensor(tf.placeholder(tf.int64, shape=[None, 2]) , tf.placeholder(tf.int32, [None]), tf.placeholder(tf.int64, [2]))
    # calc loss for batch
    self.seqLen = tf.placeholder(tf.int32, [None])
    loss = tf.nn.ctc_loss(labels=self.gtTexts, inputs=ctcIn3dTBC, sequence_length=self.seqLen, ctc_merge_repeated=True)
    # decoder: either best path decoding or beam search decoding
    if self.decoderType == DecoderType.BestPath:
      decoder = tf.nn.ctc_greedy_decoder(inputs=ctcIn3dTBC, sequence_length=self.seqLen)
    elif self.decoderType == DecoderType.BeamSearch:
      decoder = tf.nn.ctc_beam_search_decoder(inputs=ctcIn3dTBC, sequence_length=self.seqLen, beam_width=50, merge_repeated=False)
    elif self.decoderType == DecoderType.WordBeamSearch:
      # import compiled word beam search operation (see https://github.com/githubharald/CTCWordBeamSearch)
      word_beam_search_module = tf.load_op_library('TFWordBeamSearch.so')

      # prepare information about language (dictionary, characters in dataset, characters forming words) 
      chars = str().join(self.charList)
      wordChars = open(modelDir+'wordCharList.txt').read().splitlines()[0]
      corpus = open(modelDir+'corpus.txt').read()

      # decode using the "Words" mode of word beam search
      decoder = word_beam_search_module.word_beam_search(tf.nn.softmax(ctcIn3dTBC, dim=2), 50, 'Words', 0.0, corpus.encode('utf8'), chars.encode('utf8'), wordChars.encode('utf8'))

    # return a CTC operation to compute the loss and a CTC operation to decode the RNN output
    return (tf.reduce_mean(loss), decoder)


  def setupTF(self):
    "initialize TF"
    
    print('Python: '+sys.version)
    print('Tensorflow: '+tf.__version__)

    sess=tf.Session() # TF session
    
    modelDir = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/Project1NLP_Model/model/'

    saver = tf.train.Saver(max_to_keep=1) # saver saves model to file
    #modelDir = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/Project1NLP_Model/model/'
    #modelDir = '/Users/haithem.afli/Desktop/CIT/Research/IndustryProjects/Unitek/NN_OCR/run_model/model/'
    latestSnapshot = tf.train.latest_checkpoint(modelDir) # is there a saved model?

    # if model must be restored (for inference), there must be a snapshot
    if self.mustRestore and not latestSnapshot:
      raise Exception('No saved model found in: ' + modelDir)

    # load saved model if available
    if latestSnapshot:
      print('Init with stored values from ' + latestSnapshot)
      saver.restore(sess, latestSnapshot)
    else:
      print('Init with new values')
      sess.run(tf.global_variables_initializer())

    return (sess,saver)


  def toSparse(self, texts):
    "put ground truth texts into sparse tensor for ctc_loss"
    indices = []
    values = []
    shape = [len(texts), 0] # last entry must be max(labelList[i])

    # go over all texts
    for (batchElement, text) in enumerate(texts):
      # convert to string of label (i.e. class-ids)
      labelStr = [self.charList.index(c) for c in text]
      # sparse tensor must have size of max. label-string
      if len(labelStr) > shape[1]:
        shape[1] = len(labelStr)
      # put each label into sparse tensor
      for (i, label) in enumerate(labelStr):
        indices.append([batchElement, i])
        values.append(label)

    return (indices, values, shape)


  def decoderOutputToText(self, ctcOutput):
    "extract texts from output of CTC decoder"
    
    # contains string of labels for each batch element
    encodedLabelStrs = [[] for i in range(Model.batchSize)]

    # word beam search: label strings terminated by blank
    if self.decoderType == DecoderType.WordBeamSearch:
      blank=len(self.charList)
      for b in range(Model.batchSize):
        for label in ctcOutput[b]:
          if label==blank:
            break
          encodedLabelStrs[b].append(label)

    # TF decoders: label strings are contained in sparse tensor
    else:
      # ctc returns tuple, first element is SparseTensor 
      decoded=ctcOutput[0][0] 

      # go over all indices and save mapping: batch -> values
      idxDict = { b : [] for b in range(Model.batchSize) }
      for (idx, idx2d) in enumerate(decoded.indices):
        label = decoded.values[idx]
        batchElement = idx2d[0] # index according to [b,t]
        encodedLabelStrs[batchElement].append(label)

    # map labels to chars for all batch elements
    return [str().join([self.charList[c] for c in labelStr]) for labelStr in encodedLabelStrs]


  def trainBatch(self, batch):
    "feed a batch into the NN to train it"
    sparse = self.toSparse(batch.gtTexts)
    rate = 0.01 if self.batchesTrained < 10 else (0.001 if self.batchesTrained < 10000 else 0.0001) # decay learning rate
    (_, lossVal) = self.sess.run([self.optimizer, self.loss], { self.inputImgs : batch.imgs, self.gtTexts : sparse , self.seqLen : [Model.maxTextLen] * Model.batchSize, self.learningRate : rate} )
    self.batchesTrained += 1
    return lossVal


  def inferBatch(self, batch):
    "feed a batch into the NN to recngnize the texts"
    decoded = self.sess.run(self.decoder, { self.inputImgs : batch.imgs, self.seqLen : [Model.maxTextLen] * Model.batchSize } )
    return self.decoderOutputToText(decoded)
  

  def save(self):
    "save model to file"
    self.snapID += 1
    self.saver.save(self.sess, 'model/snapshot', global_step=self.snapID)
 


def infer(model, fnImg):
  "recognize text in image provided by file path"
  img = preprocess(cv2.imread(fnImg, cv2.IMREAD_GRAYSCALE), Model.imgSize)
  batch = Batch(None, [img] * Model.batchSize) # fill all batch elements with same input image
  recognized = model.inferBatch(batch) # recognize text
  print('Recognized:', '"' + recognized[0] + '"') # all batch elements hold same result
  return recognized[0]














""" The above Part has directly been taken from the lecturer code.
    Below, the main function has been modified to take the output of the lecturer code had process the solution proposed to correct the word"""





# the main function has been modified to support the proposed solution to correct the word.
def main():
  "main function"
  
  #this part has been taken from the lecturer code
  decoderType = DecoderType.BestPath

  print(open(FilePaths.fnAccuracy).read())
  model = Model(open(FilePaths.fnCharList).read(), decoderType, mustRestore=True)
  #infer(model, FilePaths.fnInfer)
  
  
  #this part below has been changed to put in place our solution.
  
  print('\n####### STARTING THE ERROR CORRECTION MODULE #######')
  
  sentence = infer(model, FilePaths.fnInfer)         # sentence take the output of the lecturer code
  print('Sentence to study:' , '"' , sentence , '"')
     
  # Below are the list of parameter we apply for our solution
  #corpus_file = '/content/drive/My Drive/Colab Notebooks/Project1NLP_Model/europarl/txt/en/all_en.txt'  # now we will consider the corpus file as input  
  ngram = 2                                                             # we are considering the bigram for our solution
  word_delta_len_modulation = 0                                         # we are not accepting any delta letter more or less than the original word (only spelling mistake is considered)
  word_similarity_acceptation = 0.5                                     # we only propose a similarity of 50% from the best proposition for our range of proposed word for correction
  word_selection_type = 'contextual'                                    # we select the best candidate word for replacement using its context in the sentence 
  
  # below we apply our methodology for word correction
  
  # dictionary is already build before the ECM test. Therefore, there is no need to create a dictionary.
  #dict_corpus = words_dict(corpus_file, 2)                              # we build our dictionnary
  
  add_word_dict('Unitek',dict_corpus,ngram)        # we add the "unitek" word in our dictionnary
  final_sentence = Error_Correction_Module(sentence, dict_corpus, ngram, word_delta_len_modulation, word_similarity_acceptation, word_selection_type) # Launching the ECM process

  print()
  print("Final Result: ", final_sentence)                               # the final result is printed
  
if __name__ == '__main__':
  main()



Validation character error rate of saved model: 13.956289%
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for up

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Sentence to expand: "Unetek"
Sentence after expansion: "Unetek"
Sentence before tokenization: "Unetek"
Sentence after tokenization: " ['Unetek'] "
Sentence before tokenization: "Unetek"
Sentence after tokenization: " ['Unetek'] "
Words candidate for correction of word  "unetek"  using ngram technique:  [('unitek', 3), ('nextek', 3), ('canete', 3), ('brunet', 3), ('retune', 3), ('tenets', 3), ('netted', 3), ('tennet', 3), ('teleki', 2), ('franek', 2), ('sponek', 2), ('petter', 2), ('rutnet', 2), ('leonet', 2), ('magnet', 2), ('steter', 2), ('yvette', 2), ('meteor', 2), ('eionet', 2), ('unamet', 2), ('tetchy', 2), ('nether', 2), ('zetter', 2), ('monnet', 2), ('chanet', 2), ('peters', 2), ('silete', 2), ('iconet', 2), ('nettle', 2), ('petten', 2), ('effete', 2), ('wetter', 2), ('cetane', 2), ('setter', 2), ('letter', 2), ('voynet', 2), ('fernet', 2), ('tonnet', 2), ('detect', 2), ('exeter', 2), ('metteg', 2), ('pieter', 2), ('meters', 2), ('etelka', 2), ('vetted', 2), ('gamete', 2), ('get

# Conclusion

The ECM implemented is relatively strong as:
* It corrects also short word mispelling
* It is able to detect the context of the mispelled word and correct it in many cases.
* And it also keep the formatting of the original sentence (with its contraction and upper case letter).

However, we have seen some limitation of our ECM model. Typically, if the Levenshtein distance of the proposed words are identical but their context are different (transitive verbs, intransitive verbs, noun, adjective, ...), the ECM is able to propose an accurate word proposal for correction. But as soon as their context get very similar (both are transitive verbs, intransitive verbs, noun, or adjective, ...), our ECM has difficulty to propose an accurate solution.

2 ways can be done to avoid such limitaion:
1. The ECM model should extend the context to a wider one (it is currently a bigram context, we should think to use for example a trigram context)
1. The ECM could also integrate a Machine Learning module to identify use cases of the proposed word for correction.

